# IMPORT DEPENDENCIES

In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

In [32]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Aakash\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aakash\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [86]:
from nltk.tokenize import word_tokenize

# LOAD INPUT FILE

In [33]:
df = pd.read_excel('Input.xlsx')
df.head()

,URL_ID,URL
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...


### We drop URL_ID and we have to loop each url and extract data using BS lets do it

In [34]:
df = df.drop('URL_ID', axis=1)

In [35]:
df.head()

,URL
0,https://insights.blackcoffer.com/rising-it-cit...
1,https://insights.blackcoffer.com/rising-it-cit...
2,https://insights.blackcoffer.com/internet-dema...
3,https://insights.blackcoffer.com/rise-of-cyber...
4,https://insights.blackcoffer.com/ott-platform-...


# EXTRACT DATA FROM EACH URL

In [49]:
# Initialize the URL ID
url_id = 1

# Assuming df is your DataFrame with URLs
for i in range(len(df)):
    j = df.iloc[i].values
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}
    page = requests.get(j[0], headers=headers)

    soup = BeautifulSoup(page.content, 'html.parser')

    # Find the content
    content = soup.findAll(attrs={'class': 'td-post-content'})
    if content:
        content_text = content[0].text.replace('\xa0', "  ").replace('\n', "  ")
    else:
        content_text = "Content not found"

    # Find the title
    title = soup.findAll(attrs={'class': 'entry-title'})
    if title:
        # Ensure there are enough elements in the title list to access the 17th element (index 16)
        if len(title) > 16:
            title_text = title[16].text.replace('\n', "  ").replace('/', "")
        else:
            title_text = "Title not found"
    else:
        title_text = "Title not found"

    # Combine title and content
    text = title_text + '.' + content_text

    # Save text to a pandas Series
    df1 = pd.Series([text])

    # Save text to a file
#     file_name = f"{url_id}.txt"
#     with open(file_name, 'w', encoding='utf-8') as f:
#         f.write(text)

    # Increment URL ID
    url_id += 1


In [50]:
type(text)

str

In [52]:
text

'AI Bot Audio to audio.    As business close to help prevent transmission of COVID-19, financial concerns and job losses are one of the first human impacts of the virus;  Not knowing how this pandemic will play out also affects our economic, physical and mental well-being;Despite this fear, businesses and communities in many regions have shown a more altruistic response in the face of crisis – actions which could help countries preparing for COVID-19.  COVID-19 is in decline in China. There are now more new cases every day in Europe than there were in China at the epidemic’s peak and Italy has surpassed it as the country with the most deaths from the virus It took 67 days to reach the first 100,000 confirmed cases worldwide, 11 days for this to increase to 200,000and just four to reach 300,000 confirmed cases – a figure now exceeded.  The Human Consequences  In recent weeks, we have seen the significant economic impact of the coronavirus on financial markets and vulnerable industries s

# CONVERT TEXT TO SENTENCE

In [57]:
# Assuming `text` is already a list or array of strings, convert it to a pandas Series
text = pd.Series(text)

# Splitting text on '. ' (dot followed by a space)
a = text.str.split(r'(?<=[.])\s', expand=False)  # splitting on '. ' using regex to keep the period with the sentence

# Explode to convert list elements to rows
b = a.explode()

# Create DataFrame
b = pd.DataFrame(b, columns=['abc'])

print(b)



                                                 abc
0                             AI Bot Audio to audio.
0     As business close to help prevent transmiss...
0                   COVID-19 is in decline in China.
0  There are now more new cases every day in Euro...
0   The Human Consequences  In recent weeks, we h...
0   Travel and tourism account for 10% of the glo...
0   Global tourism, travel and hospitality compan...
0  This, in turn, affects many people, typically ...
0  Some governments have announced economic measu...
0   Behind these statistics lie the human costs o...
0  Not knowing how this pandemic will play out af...
0   Fear of the unknown can often lead to feeling...
0   In all cases, lack of information or the wron...
0  There is a huge amount of misleading informati...
0  People are susceptible to social media posts f...
0   Sadly, criminals and hackers are also exploit...
0  Hackers are selling malware and hacking tools ...
0   Social distancing and lockdowns have also 

### removing . char from each row

In [59]:
def abcd(x):    
    nopunc =[char for char in x if char != '.']
    return ''.join(nopunc)
b['abc']=b['abc'].apply(abcd)


### replacing emty space with null values

In [60]:
c=b.replace('',np.nan,regex=True)
c=c.mask(c==" ")
c=c.dropna()
c.reset_index(drop=True,inplace=True)

In [61]:
c

,abc
0,AI Bot Audio to audio
1,As business close to help prevent transmiss...
2,COVID-19 is in decline in China
3,There are now more new cases every day in Euro...
4,"The Human Consequences In recent weeks, we h..."
5,Travel and tourism account for 10% of the glo...
6,"Global tourism, travel and hospitality compan..."
7,"This, in turn, affects many people, typically ..."
8,Some governments have announced economic measu...
9,Behind these statistics lie the human costs o...


# APPLAY STOPWORDS AND NLTK LIB

In [63]:
import string
punc=[punc for punc in string.punctuation]

In [72]:
def read_stopwords(filepath):
    with open(filepath, 'r', encoding="ISO-8859-1") as file:
        stopwords = file.read().splitlines()
    return pd.DataFrame(stopwords, columns=['stopword'])

StopWords_Auditor = read_stopwords("./StopWords-20240612T123345Z-001/StopWords/StopWords_Auditor.txt")
StopWords_Currencies = read_stopwords("./StopWords-20240612T123345Z-001/StopWords/StopWords_Currencies.txt")
StopWords_DatesandNumbers = read_stopwords("./StopWords-20240612T123345Z-001/StopWords/StopWords_DatesandNumbers.txt")
StopWords_Generic = read_stopwords("./StopWords-20240612T123345Z-001/StopWords/StopWords_Generic.txt")
StopWords_GenericLong = read_stopwords("./StopWords-20240612T123345Z-001/StopWords/StopWords_GenericLong.txt")
StopWords_Geographic = read_stopwords("./StopWords-20240612T123345Z-001/StopWords/StopWords_Geographic.txt")
StopWords_Names = read_stopwords("./StopWords-20240612T123345Z-001/StopWords/StopWords_Names.txt")


In [74]:
def text_process(text):
    nopunc =[char for char in text if char not in punc or char not in [':',',','(',')','’','?']]
    nopunc=''.join(nopunc)
    txt=' '.join([word for word in nopunc.split() if word.lower() not in StopWords_Auditor])
    txt1=' '.join([word for word in txt.split() if word.lower() not in StopWords_Currencies])
    txt2=' '.join([word for word in txt1.split() if word.lower() not in StopWords_DatesandNumbers])
    txt3=' '.join([word for word in txt2.split() if word.lower() not in StopWords_Generic])
    txt4=' '.join([word for word in txt3.split() if word.lower() not in StopWords_GenericLong])
    txt5=' '.join([word for word in txt4.split() if word.lower() not in StopWords_Geographic])
    return ' '.join([word for word in txt5.split() if word.lower() not in StopWords_Names])


In [75]:
c['abc']=c['abc'].apply(text_process)

In [77]:
c.head()

,abc
0,AI Bot Audio to audio
1,As business close to help prevent transmission...
2,COVID-19 is in decline in China
3,There are now more new cases every day in Euro...
4,The Human Consequences In recent weeks we have...


# IMPORTING MASTER DIC

In [82]:
def read_words(filepath, encoding=None):
    with open(filepath, 'r', encoding=encoding) as file:
        words = file.read().splitlines()
    return pd.DataFrame(words, columns=['word'])

positive = read_words("./MasterDictionary-20240612T123345Z-001/MasterDictionary/positive-words.txt")
negative = read_words("./MasterDictionary-20240612T123345Z-001/MasterDictionary/negative-words.txt", encoding="ISO-8859-1")

In [84]:
positive.columns=['abc']
negative.columns=['abc']
positive['abc']=positive['abc'].astype(str)
negative['abc']=negative['abc'].astype(str)
     

#positive and negative dictionary without stopwords
positive['abc']=positive['abc'].apply(text_process)
negative['abc']=negative['abc'].apply(text_process)
     

#positive list
length=positive.shape[0]
post=[]
for i in range(0,length):
   nopunc =[char for char in positive.iloc[i] if char not in string.punctuation or char != '+']
   nopunc=''.join(nopunc)

   post.append(nopunc)

In [85]:
length=negative.shape[0]
neg=[]
for i in range(0,length):
  nopunc =[char for char in negative.iloc[i] if char not in string.punctuation or char != '+']
  nopunc=''.join(nopunc)
  neg.append(nopunc)
     

# TOKENIZE WORDS

In [87]:
txt_list=[]
length=c.shape[0]
for i in range(0,length):
  txt=' '.join([word for word in c.iloc[i]])
  txt_list.append(txt)
     

#tokenization of text
tokenize_text=[]
for i in txt_list:
  
  tokenize_text+=(word_tokenize(i))

In [88]:
print(tokenize_text)

['AI', 'Bot', 'Audio', 'to', 'audio', 'As', 'business', 'close', 'to', 'help', 'prevent', 'transmission', 'of', 'COVID-19', 'financial', 'concerns', 'and', 'job', 'losses', 'are', 'one', 'of', 'the', 'first', 'human', 'impacts', 'of', 'the', 'virus', ';', 'Not', 'knowing', 'how', 'this', 'pandemic', 'will', 'play', 'out', 'also', 'affects', 'our', 'economic', 'physical', 'and', 'mental', 'well-being', ';', 'Despite', 'this', 'fear', 'businesses', 'and', 'communities', 'in', 'many', 'regions', 'have', 'shown', 'a', 'more', 'altruistic', 'response', 'in', 'the', 'face', 'of', 'crisis', '–', 'actions', 'which', 'could', 'help', 'countries', 'preparing', 'for', 'COVID-19', 'COVID-19', 'is', 'in', 'decline', 'in', 'China', 'There', 'are', 'now', 'more', 'new', 'cases', 'every', 'day', 'in', 'Europe', 'than', 'there', 'were', 'in', 'China', 'at', 'the', 'epidemic', '’', 's', 'peak', 'and', 'Italy', 'has', 'surpassed', 'it', 'as', 'the', 'country', 'with', 'the', 'most', 'deaths', 'from', 'th

# CALCULATE SCORE

In [106]:
# 1) POSITIVE SCORE
positive_score=0
for i in tokenize_text:
  if(i.lower() in post):
    positive_score+=1
print('postive score=', positive_score)


postive score= 38


In [107]:

#2) NEGATIVE SCORE
negative_score=0
for i in tokenize_text:
  if(i.lower() in neg):
    negative_score+=1
print('negative score=', negative_score)

negative score= 58


In [108]:
#3) POLARITY SCORE
#Polarity Score = (Positive Score – Negative Score)/ ((Positive Score + Negative Score) + 0.000001)
Polarity_Score=(positive_score-negative_score)/((positive_score+negative_score)+0.000001)
print('polarity_score=', Polarity_Score)

polarity_score= -0.20833333116319447


In [109]:

#4) SUBJECTIVITY SCORE
#Subjectivity Score = (Positive Score + Negative Score)/ ((Total Words after cleaning) + 0.000001)
subjectiivity_score=(positive_score-negative_score)/((len(tokenize_text))+ 0.000001)
print('subjectivity_score',subjectiivity_score)

subjectivity_score -0.01857010211831931


In [110]:

#5) AVG SENTENCE LENGTH
length=c.shape[0]
avg_length=[]
for i in range(0,length):
  avg_length.append(len(c['abc'].iloc[i]))
avg_senetence_length=sum(avg_length)/len(avg_length)
print('avg sentence length=', avg_senetence_length)
     
#6) PERCENTAGE OF COMPLEX WORDS
vowels=['a','e','i','o','u']
import re
count=0
complex_Word_Count=0
for i in tokenize_text:
  x=re.compile('[es|ed]$')
  if x.match(i.lower()):
   count+=0
  else:
    for j in i:
      if(j.lower() in vowels ):
        count+=1
  if(count>2):
   complex_Word_Count+=1
  count=0

Percentage_of_Complex_words=complex_Word_Count/len(tokenize_text)
print('percentag of complex words= ',Percentage_of_Complex_words)

#7) FOG INDEX
Fog_Index = 0.4 * (avg_senetence_length + Percentage_of_Complex_words)
print('fog index= ',Fog_Index )

#8) AVG NUMBER OF WORDS PER SENTENCE
length=c.shape[0]
avg_length=[]
for i in range(0,length):
  a=[word.split( ) for word in c.iloc[i]]
  avg_length.append(len(a[0]))
  a=0
#avg
avg_no_of_words_per_sentence=sum(avg_length)/length
print("avg no of words per sentence= ",avg_no_of_words_per_sentence)
     
#9) COMPLEX WORD COUNT
vowels=['a','e','i','o','u']
import re
count=0
complex_Word_Count=0
for i in tokenize_text:
  x=re.compile('[es|ed]$')
  if x.match(i.lower()):
   count+=0
  else:
    for j in i:
      if(j.lower() in vowels ):
        count+=1
  if(count>2):
   complex_Word_Count+=1
  count=0
print('complex words count=',  complex_Word_Count)

#10) WORD COUNT
word_count=len(tokenize_text)
print('word count= ', word_count)
     
#11) SYLLABLE PER WORD
vowels=['a','e','i','o','u']
import re
count=0
for i in tokenize_text:
  x=re.compile('[es|ed]$')
  if x.match(i.lower()):
   count+=0
  else:
    for j in i:
      if(j.lower() in vowels ):
        count+=1
syllable_count=count
print('syllable_per_word= ',syllable_count)
     

#12) PERSONAL PRONOUNS
pronouns=['i','we','my','ours','us' ]
import re
count=0
for i in tokenize_text:
  if i.lower() in pronouns:
   count+=1
personal_pronouns=count
print('personal pronouns= ',personal_pronouns )

#13) AVG WORD LENGTH
count=0
for i in tokenize_text:
  for j in i:
    count+=1
avg_word_length=count/len(tokenize_text)
print('avg word= ', avg_word_length)

avg sentence length= 199.78787878787878
percentag of complex words=  0.28040854224698236
fog index=  80.02731493205032
avg no of words per sentence=  31.545454545454547
complex words count= 302
word count=  1077
syllable_per_word=  2106
personal pronouns=  3
avg word=  5.185701021355618


## And here we got all scores

In [137]:
data={'positive_score':positive_score,'negative_score':negative_score,'Polarity_Score':Polarity_Score,'subjectiivity_score':subjectiivity_score,'avg_senetence_length':avg_senetence_length,'Percentage_of_Complex_words':Percentage_of_Complex_words,'Fog_Index':Fog_Index,'avg_no_of_words_per_sentence':avg_no_of_words_per_sentence,'complex_Word_Count':complex_Word_Count,'word_count':word_count,'syllable_count':syllable_count,'personal_pronouns':personal_pronouns,'avg_word_length':avg_word_length}
     

In [142]:


output=pd.DataFrame()
output=output.append(data,ignore_index=True)
output.columns=['positive_score','negative_score','Polarity_Score','subjectiivity_score','avg_senetence_length','Percentage_of_Complex_words','Fog_Index','avg_no_of_words_per_sentence','complex_Word_Count','word_count','syllable_count','personal_pronouns','avg_word_length']
output

C:\Users\Aakash\AppData\Local\Temp\ipykernel_4264\3814463793.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output=output.append(data,ignore_index=True)


,positive_score,negative_score,Polarity_Score,subjectiivity_score,avg_senetence_length,Percentage_of_Complex_words,Fog_Index,avg_no_of_words_per_sentence,complex_Word_Count,word_count,syllable_count,personal_pronouns,avg_word_length
0,38.0,58.0,-0.208333,-0.01857,199.787879,0.280409,80.027315,31.545455,302.0,1077.0,2106.0,3.0,5.185701


In [143]:
# with open('output.csv', 'a') as f: 
#    output.to_csv(f, index=False, header=False)
# files.download('output.csv')

NameError: name 'files' is not defined

# I HAVE DONE THIS ONLY FOR ONE TEXT FILE YOU CAN DO FOR OTHER 99\